In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import random
import networkx as nx

In [ ]:
def ODEsolve(nodes, time, InitialCondition, beta, gamma):
    sol = solve_ivp(ODE, (time[0], time[-1]), InitialCondition, t_eval = time)
    
    Fig_Data = plt.figure()

    plt.plot(sol['t'], sol['y'][0], label = 'S')
    plt.plot(sol['t'], sol['y'][1], label = 'I')
    plt.plot(sol['t'], sol['y'][2], label = 'R')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()

def ODE(time, y):
    Spos, Ipos, Rpos = y
    return [-beta*Ipos*Spos, beta*Ipos*Spos - gamma*Ipos, gamma*Ipos]

In [ ]:
# Create network and startsituation
def start(nodes, sickPerc, radius):
    Network = nx.random_geometric_graph(nodes, radius)
    Situation = []
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < sickPerc:
            Situation.append(1)
        else:
            Situation.append(0)
    return Network, Situation

## SIR with ODE
# 0 for S, 1 for I, 2 for R
def updateSIR(y, beta, gamma, Network):
    yNew = []
    for i in range(len(Network)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                if y[j] == 1:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
            else:
                yNew.append(y[i])
        elif y[i] == 1:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                yNew.append(2)
            else:
                yNew.append(y[i])
        else:
            yNew.append(y[i])
    return yNew

def spreaddiseaseSIR(NLoops, StartSituation, beta, gamma, Network):
    yList = [StartSituation]
    yNew = StartSituation
    for i in range(NLoops):
        y = yNew
        yNew = updateSIR(y, beta, gamma, Network)
        yList.append(yNew)
    return yList

def countingSIR(situations):
    S = []
    I = []
    R = []
    for i in range(len(situations)):
        S.append(np.bincount(situations[i], minlength = 3)[0])
        I.append(np.bincount(situations[i], minlength = 3)[1])
        R.append(np.bincount(situations[i], minlength = 3)[2])
    return S, I, R

def everythingSIR(nodes, sickPerc, radius, NLoops, beta, gamma):
    G, StartSituation = start(nodes, sickPerc, radius)
    situations = spreaddiseaseSIR(NLoops, StartSituation, beta, gamma, G)
    S, I, R = countingSIR(situations)
    time = np.arange(0, NLoops + 1, 1)
    Fig_Data = plt.figure()

    plt.plot(time, S, label = 'S')
    plt.plot(time, I, label = 'I')
    plt.plot(time, R, label = 'R')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()
    return G, S, I, R

def averageSIR(nodes, sickPerc, radius, NLoops, beta, gamma, N):
    time = np.arange(0, NLoops + 1, 1)
    AverageS = np.zeros(NLoops + 1)
    AverageI = np.zeros(NLoops + 1)
    AverageR = np.zeros(NLoops + 1)
    for i in range(N):
        G, StartSituation = start(nodes, sickPerc, radius)
        situations = spreaddiseaseSIR(NLoops, StartSituation, beta, gamma, G)
        S, I, R = countingSIR(situations)
        AverageS += S
        AverageI += I
        AverageR += R
    AverageS = AverageS/N
    AverageI = AverageI/N
    AverageR = AverageR/N
    
    Fig_Data = plt.figure()

    plt.plot(time, AverageS, label = 'S')
    plt.plot(time, AverageI, label = 'I')
    plt.plot(time, AverageR, label = 'R')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()
    
    plt.show()

In [ ]:
nodes = 200
radius = 1
NLoops = 40
time = np.arange(0, NLoops, 1)
sickPerc = 0.2
InitialCondition = [0.8*nodes, 0.2*nodes, 0]
N = 100

beta = 0.3/nodes
gamma = 0.05

In [ ]:
ODEsolve(nodes, time, InitialCondition, beta, gamma)

In [ ]:
averageSIR(nodes, sickPerc, radius, NLoops, beta, gamma, N)